In [1]:
!pip install pyspark py4j

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.master("local[*]").getOrCreate()
spark

23/10/26 21:55:50 WARN Utils: Your hostname, Archits-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.7 instead (on interface en0)
23/10/26 21:55:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/26 21:55:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
import pandas as pd
pd_df=pd.read_csv("WMT.csv")
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1259 non-null   object 
 1   Open       1259 non-null   float64
 2   High       1259 non-null   float64
 3   Low        1259 non-null   float64
 4   Close      1259 non-null   float64
 5   Adj Close  1259 non-null   float64
 6   Volume     1259 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 69.0+ KB


In [6]:
pd_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-01-20,61.799999,62.330002,60.200001,60.840000,53.990601,17369100
1,2016-01-21,60.980000,62.790001,60.910000,61.880001,54.913509,12089200
2,2016-01-22,62.439999,63.259998,62.130001,62.689999,55.632324,9197500
3,2016-01-25,62.779999,63.820000,62.549999,63.450001,56.306763,12823400
4,2016-01-26,63.360001,64.470001,63.259998,64.000000,56.794834,9441200


In [5]:
df= spark.read.csv("WMT.csv",inferSchema=True,header=True)
df.head()

Row(Date=datetime.date(2016, 1, 20), Open=61.799999, High=62.330002, Low=60.200001, Close=60.84, Adj Close=53.990601, Volume=17369100)

In [8]:
df.show()

+----------+---------+---------+---------+---------+---------+--------+
|      Date|     Open|     High|      Low|    Close|Adj Close|  Volume|
+----------+---------+---------+---------+---------+---------+--------+
|2016-01-20|61.799999|62.330002|60.200001|    60.84|53.990601|17369100|
|2016-01-21|    60.98|62.790001|    60.91|61.880001|54.913509|12089200|
|2016-01-22|62.439999|63.259998|62.130001|62.689999|55.632324| 9197500|
|2016-01-25|62.779999|    63.82|62.549999|63.450001|56.306763|12823400|
|2016-01-26|63.360001|64.470001|63.259998|     64.0|56.794834| 9441200|
|2016-01-27|64.099998|    65.18|63.889999|63.950001|56.750477|10214300|
|2016-01-28|64.029999|64.510002|    63.43|64.220001| 56.99007|11278300|
|2016-01-29|    64.75|66.529999|64.739998|66.360001|58.889149|16439100|
|2016-02-01|65.910004|    67.93|65.889999|     67.5| 59.90081|14728400|
|2016-02-02|67.300003|67.839996|66.279999|66.860001|59.332867|13585900|
|2016-02-03|67.309998|     67.5|    65.07|66.269997| 58.80928|12

In [10]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [11]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

In [12]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [14]:
df.describe().show()

23/10/26 22:08:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+----------------+------------------+------------------+------------------+------------------+-----------------+
|summary|            Open|              High|               Low|             Close|         Adj Close|           Volume|
+-------+----------------+------------------+------------------+------------------+------------------+-----------------+
|  count|            1259|              1259|              1259|              1259|              1259|             1259|
|   mean|96.5083001715647| 97.33101670611597| 95.74480543367744| 96.54861796902313| 92.42759966243042|8509255.043685464|
| stddev|23.3274864439079|23.590538442427473|23.019708825126568|23.292869033013766|25.335281463390604|4760478.447370805|
|    min|           60.98|         62.330002|         60.200001|             60.84|         53.990601|          2227400|
|    max|      153.600006|        153.660004|        151.660004|        152.789993|        152.233536|         56233000|
+-------+----------------+------

In [15]:
df.filter('Close>60').show()

+----------+---------+---------+---------+---------+---------+--------+
|      Date|     Open|     High|      Low|    Close|Adj Close|  Volume|
+----------+---------+---------+---------+---------+---------+--------+
|2016-01-20|61.799999|62.330002|60.200001|    60.84|53.990601|17369100|
|2016-01-21|    60.98|62.790001|    60.91|61.880001|54.913509|12089200|
|2016-01-22|62.439999|63.259998|62.130001|62.689999|55.632324| 9197500|
|2016-01-25|62.779999|    63.82|62.549999|63.450001|56.306763|12823400|
|2016-01-26|63.360001|64.470001|63.259998|     64.0|56.794834| 9441200|
|2016-01-27|64.099998|    65.18|63.889999|63.950001|56.750477|10214300|
|2016-01-28|64.029999|64.510002|    63.43|64.220001| 56.99007|11278300|
|2016-01-29|    64.75|66.529999|64.739998|66.360001|58.889149|16439100|
|2016-02-01|65.910004|    67.93|65.889999|     67.5| 59.90081|14728400|
|2016-02-02|67.300003|67.839996|66.279999|66.860001|59.332867|13585900|
|2016-02-03|67.309998|     67.5|    65.07|66.269997| 58.80928|12

In [16]:
df.filter(df['Close']>60).show()

+----------+---------+---------+---------+---------+---------+--------+
|      Date|     Open|     High|      Low|    Close|Adj Close|  Volume|
+----------+---------+---------+---------+---------+---------+--------+
|2016-01-20|61.799999|62.330002|60.200001|    60.84|53.990601|17369100|
|2016-01-21|    60.98|62.790001|    60.91|61.880001|54.913509|12089200|
|2016-01-22|62.439999|63.259998|62.130001|62.689999|55.632324| 9197500|
|2016-01-25|62.779999|    63.82|62.549999|63.450001|56.306763|12823400|
|2016-01-26|63.360001|64.470001|63.259998|     64.0|56.794834| 9441200|
|2016-01-27|64.099998|    65.18|63.889999|63.950001|56.750477|10214300|
|2016-01-28|64.029999|64.510002|    63.43|64.220001| 56.99007|11278300|
|2016-01-29|    64.75|66.529999|64.739998|66.360001|58.889149|16439100|
|2016-02-01|65.910004|    67.93|65.889999|     67.5| 59.90081|14728400|
|2016-02-02|67.300003|67.839996|66.279999|66.860001|59.332867|13585900|
|2016-02-03|67.309998|     67.5|    65.07|66.269997| 58.80928|12

In [22]:
df.na.df.show()

+----------+---------+---------+---------+---------+---------+--------+
|      Date|     Open|     High|      Low|    Close|Adj Close|  Volume|
+----------+---------+---------+---------+---------+---------+--------+
|2016-01-20|61.799999|62.330002|60.200001|    60.84|53.990601|17369100|
|2016-01-21|    60.98|62.790001|    60.91|61.880001|54.913509|12089200|
|2016-01-22|62.439999|63.259998|62.130001|62.689999|55.632324| 9197500|
|2016-01-25|62.779999|    63.82|62.549999|63.450001|56.306763|12823400|
|2016-01-26|63.360001|64.470001|63.259998|     64.0|56.794834| 9441200|
|2016-01-27|64.099998|    65.18|63.889999|63.950001|56.750477|10214300|
|2016-01-28|64.029999|64.510002|    63.43|64.220001| 56.99007|11278300|
|2016-01-29|    64.75|66.529999|64.739998|66.360001|58.889149|16439100|
|2016-02-01|65.910004|    67.93|65.889999|     67.5| 59.90081|14728400|
|2016-02-02|67.300003|67.839996|66.279999|66.860001|59.332867|13585900|
|2016-02-03|67.309998|     67.5|    65.07|66.269997| 58.80928|12

In [18]:
df=df.na.drop(how="all")

In [25]:
df.na.drop(how="all")

DataFrame[Date: date, Open: double, High: double, Low: double, Close: double, Adj Close: double, Volume: int]

In [20]:
df.show()

+----------+---------+---------+---------+---------+---------+--------+
|      Date|     Open|     High|      Low|    Close|Adj Close|  Volume|
+----------+---------+---------+---------+---------+---------+--------+
|2016-01-20|61.799999|62.330002|60.200001|    60.84|53.990601|17369100|
|2016-01-21|    60.98|62.790001|    60.91|61.880001|54.913509|12089200|
|2016-01-22|62.439999|63.259998|62.130001|62.689999|55.632324| 9197500|
|2016-01-25|62.779999|    63.82|62.549999|63.450001|56.306763|12823400|
|2016-01-26|63.360001|64.470001|63.259998|     64.0|56.794834| 9441200|
|2016-01-27|64.099998|    65.18|63.889999|63.950001|56.750477|10214300|
|2016-01-28|64.029999|64.510002|    63.43|64.220001| 56.99007|11278300|
|2016-01-29|    64.75|66.529999|64.739998|66.360001|58.889149|16439100|
|2016-02-01|65.910004|    67.93|65.889999|     67.5| 59.90081|14728400|
|2016-02-02|67.300003|67.839996|66.279999|66.860001|59.332867|13585900|
|2016-02-03|67.309998|     67.5|    65.07|66.269997| 58.80928|12

In [23]:
df.select('Open').show()

+---------+
|     Open|
+---------+
|61.799999|
|    60.98|
|62.439999|
|62.779999|
|63.360001|
|64.099998|
|64.029999|
|    64.75|
|65.910004|
|67.300003|
|67.309998|
|65.760002|
|66.860001|
|     66.5|
|65.489998|
|66.190002|
|65.019997|
|65.519997|
|66.610001|
|66.099998|
+---------+
only showing top 20 rows



In [24]:
df.select('Open').distinct().show()

+----------+
|      Open|
+----------+
|     60.98|
| 71.309998|
| 71.599998|
|      74.5|
|     70.07|
| 70.870003|
| 76.080002|
| 79.620003|
| 96.849998|
|     95.57|
| 97.940002|
|132.589996|
|    151.25|
| 84.139999|
| 95.470001|
|107.019997|
|123.900002|
|      67.0|
| 73.730003|
| 72.309998|
+----------+
only showing top 20 rows



In [26]:
from pyspark.sql import functions as F

In [27]:
df.groupBy("Date").agg(F.sum("Open")).show()

+----------+----------+
|      Date| sum(Open)|
+----------+----------+
|2016-03-01| 66.650002|
|2017-08-11| 80.730003|
|2017-09-11| 79.150002|
|2018-08-10| 88.699997|
|2019-05-08|    100.25|
|2019-06-04|102.610001|
|2020-08-24|    132.25|
|2016-04-25| 68.550003|
|2017-01-06| 68.410004|
|2018-06-06| 84.949997|
|2019-11-01|117.919998|
|2019-11-18|118.449997|
|2020-01-21|114.589996|
|2020-07-24|130.949997|
|2020-08-05|130.449997|
|2016-05-03| 66.860001|
|2016-07-26| 73.709999|
|2016-08-15| 73.809998|
|2016-08-31| 71.459999|
|2016-10-03| 71.830002|
+----------+----------+
only showing top 20 rows



In [28]:
df.write.csv("output")

In [29]:
from pyspark import SparkContext

In [30]:
sc=SparkContext.getOrCreate()

In [31]:
rdd_obj=df.rdd
print(type(rdd_obj))

<class 'pyspark.rdd.RDD'>


In [32]:
rdd_obj.getNumPartitions()

1

In [34]:
new_rdd=rdd_obj.repartition(5)
print(new_rdd.getNumPartitions())

5


In [35]:
numbers = list(range(1000000))

In [36]:
%time
res=filter(lambda i: i % 2,numbers)

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 7.87 µs


In [37]:
RDD_example = sc.parallelize(numbers)
print(RDD_example.getNumPartitions())

8


In [42]:
%time
o = RDD_example.filter(lambda i: i % 2)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 12.2 µs
